In [1]:
# Entraînement du modèle
#model.learn(total_timesteps=10000)

# Sauvegarde du modèle
#model.save("dqn_pokemon_showdown")

# Charger le modèle plus tard
# model = DQN.load("dqn_pokemon_showdown", env=train_env)

In [2]:
import inspect
from poke_env.player import RandomPlayer

print(inspect.getsource(RandomPlayer))


class RandomPlayer(Player):
    def choose_move(self, battle: AbstractBattle) -> BattleOrder:
        return self.choose_random_move(battle)



In [3]:
from poke_env.player import Player
import torch
from stable_baselines3 import DQN
from poke_env.data import GenData
import numpy as np

# Initialiser GenData pour la génération souhaitée (par exemple, génération 8)
gen_data = GenData.from_gen(8)

# Accéder au tableau des types
type_chart = gen_data.type_chart

class DQ_simple(Player):
    def __init__(self,account_configuration,model_path = "dqn_pokemon_showdown", battle_format="gen8randombattle") :
        super().__init__(battle_format=battle_format,account_configuration=account_configuration)

        # Charger le modèle DQN
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print("📥 Modèle DQN chargé :", self.model)

    def choose_move(self, battle):
        print("👉 choose_move appelée !")
        # 🔍 Debug : Voir les moves disponibles
        print(f"🔍 Moves disponibles : {[move.id for move in battle.available_moves]}")

        # Obtenir l'observation de l'état du combat
        obs = self.embed_battle(battle)
        print("📊 Observation de l'état :", obs)

        # Transformer en format PyTorch
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        print("📊 Tensor pour le modèle :", obs_tensor)

        # Prédire l'action avec le modèle DQN
        action = int(self.model.predict(obs_tensor, deterministic=True)[0])
        print("🎯 Action choisie par DQN :", action)
        if 0 <= action < len(battle.available_moves):
            move = battle.available_moves[action]
            print(f"✅ Move choisi : {move.id}")

            order = self.create_order(move)
            print(f"📤 Ordre créé : {order}")  # Debug : voir l'ordre exact généré

            return order
        else:
            print(f"❌ Action {action} invalide, on joue un move aléatoire !")
            return self.choose_random_move(battle)
    
    def embed_battle(self, battle):
        """Transforme l'état du combat en une entrée utilisable par le modèle DQN."""
        moves_base_power = np.zeros(4)
        moves_dmg_multiplier = np.ones(4)

        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = move.base_power / 100 if move.base_power else 0
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )

        fainted_mon_team = len([mon for mon in battle.team.values() if mon.fainted]) / 6
        fainted_mon_opponent = len([mon for mon in battle.opponent_team.values() if mon.fainted]) / 6

        obs = np.concatenate([moves_base_power, moves_dmg_multiplier, [fainted_mon_team, fainted_mon_opponent]])
        return np.float32(obs)


In [4]:
from poke_env import AccountConfiguration
account_conf = AccountConfiguration("DQSimplePlayer", "password")

bott_player = DQ_simple(account_configuration=account_conf)

📥 Modèle DQN chargé : <stable_baselines3.dqn.dqn.DQN object at 0x13ea18050>


In [5]:
await bott_player.send_challenges("[NAME]", n_challenges=1)

👉 choose_move appelée !
🔍 Moves disponibles : ['closecombat', 'machpunch', 'facade', 'knockoff']
📊 Observation de l'état : [1.2  0.4  0.7  0.65 0.5  0.5  1.   0.5  0.   0.  ]
📊 Tensor pour le modèle : tensor([[1.2000, 0.4000, 0.7000, 0.6500, 0.5000, 0.5000, 1.0000, 0.5000, 0.0000,
         0.0000]])
🎯 Action choisie par DQN : 0
✅ Move choisi : closecombat
📤 Ordre créé : /choose move closecombat


/var/folders/h3/sk18hvh55gqbml8209nmkrlc0000gq/T/ipykernel_47133/472182631.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  action = int(self.model.predict(obs_tensor, deterministic=True)[0])


👉 choose_move appelée !
🔍 Moves disponibles : ['closecombat', 'machpunch', 'facade', 'knockoff']
📊 Observation de l'état : [1.2  0.4  0.7  0.65 0.5  0.5  1.   0.5  0.   0.  ]
📊 Tensor pour le modèle : tensor([[1.2000, 0.4000, 0.7000, 0.6500, 0.5000, 0.5000, 1.0000, 0.5000, 0.0000,
         0.0000]])
🎯 Action choisie par DQN : 0
✅ Move choisi : closecombat
📤 Ordre créé : /choose move closecombat
👉 choose_move appelée !
🔍 Moves disponibles : []
📊 Observation de l'état : [0.         0.         0.         0.         1.         1.
 1.         1.         0.16666667 0.        ]
📊 Tensor pour le modèle : tensor([[0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.1667,
         0.0000]])
🎯 Action choisie par DQN : 7
❌ Action 7 invalide, on joue un move aléatoire !
👉 choose_move appelée !
🔍 Moves disponibles : ['shellsmash', 'hydropump', 'iciclespear', 'rockblast']
📊 Observation de l'état : [0.         1.1        0.25       0.25       1.         1.
 2.         1.         0.166666